In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from superpoint.models.superpoint_pytorch import SuperPoint
from accelerated_features.modules.xfeat import XFeat
from accelerated_features.modules.lighterglue import LighterGlue
import cv2
import numpy as np
from utils import generate_random_keypoints, get_descriptors_from_keypoints, cosine_sim

/home/navms-pdi2/anaconda3/envs/extractors/lib/python3.10/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
xfeat = XFeat()

loading weights from: /home/navms-pdi2/Documentos/lucas-benedetti/01-feature-extraction/accelerated_features/modules/../weights/xfeat.pt


In [5]:
reference_img = Image.open('UFRGS-01-2017.png').convert("L")
reference_input = {"image": transforms.ToTensor()(reference_img).unsqueeze(0)}
num_kpts = 2000
feats0 = xfeat.detectAndCompute(reference_input["image"], top_k = num_kpts)[0]
feats1 = get_descriptors_from_keypoints(model=xfeat, x=reference_input["image"], feats=feats0, inference=False) # (N, 64)

In [6]:
print(f"feats0['keypoints']: ",feats0['keypoints'].shape)
print(f"feats1['keypoints']: ",feats1['keypoints'].shape)
print(f"feats0['scores']: ",feats0['scores'].shape)
print(f"feats1['scores']: ",feats1['scores'].shape)
print(f"feats0['descriptors']: ",feats0['descriptors'].shape)
print(f"feats1['descriptors']: ",feats1['descriptors'].shape)


feats0['keypoints']:  torch.Size([2000, 2])
feats1['keypoints']:  torch.Size([2000, 2])
feats0['scores']:  torch.Size([2000])
feats1['scores']:  torch.Size([2000])
feats0['descriptors']:  torch.Size([2000, 64])
feats1['descriptors']:  torch.Size([2000, 64])


In [7]:
for i in range(num_kpts):
    sim = cosine_sim(feats0['descriptors'][i], feats1['descriptors'][i])
    #print(f"[{i}] cos_similarity:{sim})")
    if sim < 0.99:
        print(i)

In [10]:
scene_img = Image.open('009-align.jpg').convert("L")
scene_input = {"image": transforms.ToTensor()(scene_img).unsqueeze(0)}

In [19]:
def cropassion(img):
    # Carregar e converter para tons de cinza
    reference_img = Image.open(img).convert("L")

    # Converter para numpy
    img_np = np.array(reference_img)

    # Altura atual e nova altura
    h, w = img_np.shape
    new_h = 421

    # Largura proporcional
    scale = new_h / h
    new_w = int(w * scale)

    # Resize proporcional
    resized_img = reference_img.resize((new_w, new_h), Image.BICUBIC)
    resized_np = np.array(resized_img)

    # ---- CROP CENTRAL PARA 421 x 421 ----
    target_size = 421

    # calcular início e fim do corte horizontal
    excess = new_w - target_size
    left = excess // 2
    right = left + target_size

    crop_np = resized_np[:, left:right]
    return crop_np
crop_np = cropassion('009-align.jpg')
scene_input = {"image": transforms.ToTensor()(crop_np).unsqueeze(0)} 

In [27]:
particles_kpts = generate_random_keypoints((4800, 4800), num_keypoints=1, operation=("localized", (2860, 1700), 1000), device="cuda") #operation='random'
gt_kpt = torch.tensor([[[2860, 1700]]], device='cuda:0')
particles_kpts = torch.cat((gt_kpt, particles_kpts), dim=1)

feats0 = {'keypoints': particles_kpts[0]}
feats0 = get_descriptors_from_keypoints(model=xfeat, x=reference_input["image"], feats=feats0) # (N, 64)
feats0.update({'image_size': (reference_input['image'][0][0].shape[1], reference_input['image'][0][0].shape[0])})

feats1 = xfeat.detectAndCompute(scene_input["image"], top_k = 1)[0]
#feats1 = xfeat.computeGlobalDescriptor(scene_input["image"], resize_to_receptive=False)[0]
feats1.update({'image_size': (reference_input['image'][0][0].shape[1], reference_input['image'][0][0].shape[0])})
feats1['descriptors'] = feats1['descriptors'].repeat(particles_kpts.shape[1], 1)
feats1['keypoints'] = particles_kpts[0]

In [28]:
mkpts_0, mkpts_1, _, output = xfeat.match_lighterglue(feats0, feats1)

In [30]:
print(mkpts_0, mkpts_1)

[[2481.174   734.6561]] [[2860. 1700.]]


In [ ]:


particles_kpts = generate_random_keypoints((4800, 4800), num_keypoints=1, operation=("localized", (2860, 1700), 1000), device="cuda") #operation='random'
gt_kpt = torch.tensor([[[2860, 1700]]], device='cuda:0')
particles_kpts = torch.cat((gt_kpt, particles_kpts), dim=1)

feats1 = xfeat.detectAndCompute(reference_input["image"], top_k = 1)[0]
feats1['keypoints'] = gt_kpt[0] #1275 1722
#feats1['descriptors'] = feats1['descriptors'].repeat(particles_kpts.shape[1], 1)
desc = xfeat.computeGlobalDescriptor(reference_input["image"], resize_to_receptive=False)[0]
desc['descriptors'] = desc['descriptors'].repeat(particles_kpts.shape[1], 1)
desc['keypoints'] = particles_kpts[0]

feats0.update({'image_size': (scene_input['image'][0][0].shape[1], scene_input['image'][0][0].shape[0])})
feats1.update({'image_size': (scene_input['image'][0][0].shape[1], scene_input['image'][0][0].shape[0])})
desc.update({'image_size': (scene_input['image'][0][0].shape[1], scene_input['image'][0][0].shape[0])})
mkpts_0, mkpts_1, _, output = xfeat.match_lighterglue(feats0, desc)
